# **비지도 학습**
1. 차원축소
2. 군집: 비슷한 샘플을 클러스터로 모음
3. 이상치 탐지: '정상' 데이터가 어떻게 보이는지 학습 후 비정상 샘플 감지하는데 사용
4. 밀도 추정: 데이터셋 생성 확률 과정의 확률밀도함수 추정  
  - 밀도가 매우 낮은 영역에 놓인 샘플이 이상치일 가능성이 높아 이상치 탐지에 널리 사용
  - 데이터 분석과 시각화에도 유용

- k-평균,DBSCAN,가우시안혼합모델

## **9.1 군집**
-비슷한 샘플을 구별해 하나의 클러스터 또는 비슷한 샘플의 그룹으로 할당   
-비지도학습  
-다양한 애플리케이션에서 사용  
:고객 분류(추천 시스템)  
 데이터 분석  
 차원 축소 기법(각 클러스터에 대한 샘플의 친화성 측정 가능)  
 이상치 탐지(제조 분야에서 결함 감지,부정거래 방지)  
 준지도 학습  
 검색 엔진(제시된 이미지와 비슷한 이미지 제시)  
 이미지 분할(색을 기반으로 픽셀을 클러스터로 모아, 각 픽셀의 색을 클러스터의 평균 색으로 변환/물체 탐지 및 추적 시스템)

 -알고리즘이 다르면 다른 종류의 클러스터 감지

### **9.1.1 k-평균**
-반복 몇 번으로 데이터셋을 빠르고 효율적으로 클러스터로 묶을 수 있음  
-각 샘플의 레이블: 알고리즘이 샘플에 할당한 클러스터의 인덱스  
-클러스터의 크기가 많이 다르면 잘 작동x  
: 샘플을 클러스터에 할당할 때 센트로이드까지 거리를 고려하는 것이 전부이기 때문  
-**하드 군집**: 샘플을 하나의 클러스터에 할당  
-**소프트 군집**: 클러스터마다 샘플에 점수 부여/점수는 샘플과 센트로이드 사이의 거리 또는 유사도 점수  
-고차원 데이터셋을 샘플과 각 센트로이드 사이의 거리를 이용해 변환하면 k-차원 데이터셋이 만들어짐  
-> 매우 효율적인 비선형 차원 축소 기법
    



In [ ]:
import numpy as np
from sklearn.datasets import make_blobs

blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [ ]:
from sklearn.cluster import KMeans
k=5
kmeans=KMeans(n_clusters=k)
y_pred=kmeans.fit_predict(X)

In [ ]:
y_pred

array([4, 1, 2, ..., 3, 2, 1], dtype=int32)

In [ ]:
y_pred is kmeans.labels_

True

In [ ]:
#센트로이드 Centroid
kmeans.cluster_centers_

array([[-1.46679593,  2.28585348],
       [-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-2.79290307,  2.79641063],
       [-2.80037642,  1.30082566]])

In [ ]:
X_new=np.array([[0,2],[3,2],[-3,3],[-3,2.5]])
kmeans.predict(X_new)

array([2, 2, 3, 3], dtype=int32)

In [ ]:
#샘플과 각 센트로이드 사이의 거리
#고차원 데이터셋을 이런 방식으로 변환하면 k-차원 데이터셋이 만들어짐
#매우 효율적인 비선형 차원 축소 기법
kmeans.transform(X_new)

array([[1.49439034, 2.81093633, 0.32995317, 2.9042344 , 2.88633901],
       [4.4759332 , 5.80730058, 2.80290755, 5.84739223, 5.84236351],
       [1.69136631, 1.21475352, 3.29399768, 0.29040966, 1.71086031],
       [1.54808703, 0.72581411, 3.21806371, 0.36159148, 1.21567622]])

-**k-평균 알고리즘**  
  - 센트로이드가 주어진다면, 데이터셋에 있는 모든 샘플에 가장 가까운 센트로이드의 클러스터 할당  
  - 레이블이나 센트로이드가 주어지지 않는다면?  
  : 처음에는 센트로이드 랜덤하게 선정 > 샘플에 레이블 할당 > 센트로이드 업데이트 > 샘플에 레이블 할당 > 센트로이드 업데이트  
  하는 방식으로 센트로이드에 변화가 없을 때까지 계속 진행
  - 제한된 횟수 안에 수렴함
  - 적절한 솔루션이 아닌 지역 최적점으로 수렴하지 못할 수도 있음  

-**센트로이드 초기화 방법**  
  1. 센트로이드 위치를 근사하게 알 수 있다면(다른 군집 알고리즘 먼저 실행) init 매개변수에 센트로이드 리스트를 담은 넘파이 배열을 지정하고 n_init을 1로 설정
  2. 랜덤 초기화를 다르게 해 여러 번 알고리즘을 실행하고 가장 좋은 솔루션을 선택하는 것
    - 랜덤 초기화 횟수: n_init
    - 기본값 10
    - 사이킷런은 이 중 최선의 솔루션 반환
    - 최선의 솔루션을 찾아내기 위한 성능 지표   
    : 이너셔(각 샘플과 가장 가까운 센트로이드 사이의 평균 제곱 거리) inertia_
    
  

In [ ]:
good_init=np.array([[-3,3],[-3,2],[-1,-2],[0,2]])
kmeans=KMeans(n_clusters=5,init=good_init,n_init=1)

In [ ]:
kmeans.fit(X)
kmeans.inertia_

In [ ]:
#이너셔의 음수 값
kmeans.score()

-**k-means++**
  - 다른 센트로이드와 거리가 먼 센트로이드를 선택하는 초기화 단계
  - 최적이 아닌 솔루션으로 수렴할 가증성 크게 낮춤
  - kmeans 클래스는 기본적으로 이 방법 사용
  - 원래의 랜덤 방식 사용하려면: init="random"

-**k-평균 속도 개선과 미니배치 k-평균**
- 삼각부등식을 사용해 불필요한 계산 줄였음
- 전체 데이터셋을 사용해 반복하지 않고 각 반복마다 미니배치를 사용해 센트로이드를 조금씩 이동
- 메모리에 들어가지 않는 대량의 데이터셋에 군집 알고리즘 적용 가능 MiniBatchKMeans
  - memmap도 가능
  - 미니배치 k-평균 알고리즘이 빠르지만, 이너셔는 일반적으로 조금 더 나쁨


In [ ]:
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans=MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

MiniBatchKMeans(n_clusters=5)

-**최적의 클러스터 개수 찾기**
- 이너셔는 k가 증가함에 따라 점점 작아지므로 k를 선택할 때 좋은 성능 지표가 아님
- 실루엣 점수(모든 샘플에 대한 실루엣 계수의 평균) 이용
  - -1에서 1 사이
  - +1에 가까우면, 자신의 클러스터 안에 잘 속해있고 다른 클러스터와 멀리 떨어져 있다는 의미, 0에 가까우면 클러스터 경계에 위치한다는 의미, -1에 가까우면 이 샘플이 잘못된 클러스터에 할당되었다는 의미
  - 실루엣 다이어그램(모든 샘플의 실루엣 계수를 할당된 클러스터와 계숫값으로 정렬한 그림)

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X,kmeans.labels_)

### **9.1.2 k-평균의 한계**
-최적이 아닌 솔루션을 피하려면 알고리즘을 여러 번 실행해야함  
-클러스터 개수 지정해야함  
-클러스터의 크기나 밀집도가 서로 다르거나 원형이 아닐경우 잘 작동하지 않음(타원형은 가우시안 혼합 모델이 좋음)

-**데이터에 따라 잘 수행할 수 있는 군집 알고리즘이 다름**  
-k-평균을 실행하기 전에 입력 특성의 스케일을 맞추는 것이 중요



### **9.1.3 군집을 사용한 이미지 분할**
-이미지 분할: 이미지를 세그먼트 여러 개로 분할하는 작업  
-시맨틱 분할:동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트에 할당

In [ ]:
# 무당벌레 이미지 다운로드
import os
import urllib.request

PROJECT_ROOT_DIR = "."
images_path = os.path.join(PROJECT_ROOT_DIR, "images", "unsupervised_learning")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
filename = "ladybug.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/unsupervised_learning/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

('./images/unsupervised_learning/ladybug.png',
 <http.client.HTTPMessage at 0x7f04ce884910>)

In [ ]:
#색상분할
from matplotlib.image import imread
import os
image=imread(os.path.join("images","unsupervised_learning","ladybug.png"))
image.shape #높이,너비,컬러 채널 개수


(533, 800, 3)

In [ ]:
X=image.reshape(-1,3)
kmeans=KMeans(n_clusters=8).fit(X)
segmented_img=kmeans.cluster_centers_[kmeans.labels_]
segmented_img=segmented_img.reshape(image.shape)

### **9.1.4 군집을 사용한 전처리**
-군집은 차원축소에 효과적인 방법  
-지도 학습 알고리즘을 적용하기 전 전처리 단계로 사용가능 

In [ ]:
from sklearn.datasets import load_digits
X_digits,y_digits=load_digits(return_X_y=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_digits,y_digits)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [ ]:
log_reg.score(X_test,y_test)

0.9822222222222222

In [ ]:
#k-평균을 전처리 단계로 사용해 더 좋아지는지 확인
from sklearn.pipeline import Pipeline
pipeline=Pipeline([
                   ("kmeans",KMeans(n_clusters=50)),
                   ("log_reg",LogisticRegression())
])
pipeline.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [ ]:
pipeline.score(X_test,y_test)

0.98

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid=dict(kmeans__n_clusters=range(2,100))
grid_clf=GridSearchCV(pipeline,param_grid,cv=3,verbose=2)
grid_clf.fit(X_train,y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ...............................kmeans__n_clusters=2; total time=   0.3s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ...............................kmeans__n_clusters=2; total time=   0.4s


...

[CV] END ..............................kmeans__n_clusters=99; total time=   1.9s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV] END ..............................kmeans__n_clusters=99; total time=   1.6s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [ ]:
grid_clf.best_params_

{'kmeans__n_clusters': 65}

In [ ]:
grid_clf.score(X_test,y_test)

0.9688888888888889

### **9.1.5 군집을 사용한 준지도 학습**
-군집을 사용하는 또 다른 사례  
-레이블이 없는 데이터가 많고 레이블이 있는 데이터는 적을 때 사용

In [ ]:
n_labeled=50
log_reg=LogisticRegression()
log_reg.fit(X_train[:n_labeled],y_train[:n_labeled])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [ ]:
log_reg.score(X_test,y_test)

0.8622222222222222

In [ ]:
#대표 이미지 찾기
k=50
kmeans=KMeans(n_clusters=k)
X_digits_dist=kmeans.fit_transform(X_train)
representative_digit_idx=np.argmin(X_digits_dist,axis=0)
X_representative_digits=X_train[representative_digit_idx]

In [ ]:
y_representative_digits=np.array([
                4,8,0,6,8,3,7,7,9,2,
                5,5,8,5,2,1,2,9,6,2,
                4,6,9,0,8,3,0,7,4,1,
                6,5,2,4,1,8,6,3,9,2,
                4,2,9,4,7,6,2,3,1,1
])

In [ ]:
log_reg=LogisticRegression()
log_reg.fit(X_representative_digits,y_representative_digits)
log_reg.score(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.057777777777777775

In [ ]:
y_train_propagated=np.empty(len(X_train),dtype=np.int32)
for i in range(k):
  y_train_propagated[kmeans.labels_==i]=y_representative_digits[i]

In [ ]:
log_reg=LogisticRegression()
log_reg.fit(X_train,y_train_propagated)
log_reg.score(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.04666666666666667

In [ ]:
#레이블 부분 전파

percentile_closest=20
X_cluster_dist=X_digits_dist[np.arange(len(X_train)),kmeans.labels_]
for i in range(k):
  in_cluster=(kmeans.labels_==i)
  cluster_dist=X_cluster_dist[in_cluster]
  cutoff_distance=np.percentile(cluster_dist,percentile_closest)
  above_cutoff=(X_cluster_dist>cutoff_distance)
  X_cluster_dist[in_cluster&above_cutoff]=-1

partially_propagated=(X_cluster_dist!=-1)
X_train_partially_propagated=X_train[partially_propagated]
y_train_partially_propagated=y_train_propagated[partially_propagated]

In [ ]:
log_reg=LogisticRegression()
log_reg.fit(X_train_partially_propagated,y_train_partially_propagated)
log_reg.score(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.06666666666666667

In [ ]:
np.mean(y_train_partially_propagated==y_train[partially_propagated])

0.0657439446366782

-**능동 학습**
  - 학습 알고리즘과 상호작용하여 알고리즘이 요청할 때 특정 샘플의 레이블 제공
  - 불확실성 샘플링이 가장 널리 사용
  


### **9.1.6 DBSCAN**
-모든 클러스터가 충분히 밀집되어 있고 밀집되지 않은 지역과 잘 구분될 때 좋은 성능  
-predict() 메서드를 제공하지 않고 fit_predict() 메서드 제공, 새로운 샘플에 대해 클러스터 예측불가   
-클러스터의 모양과 개수에 상관없이 감지할 수 있음  
-이상치에 안정적이고 하이퍼파라미터가 2개뿐  
-클러스터 간의 밀집도가 크게 다르면 모든 클러스터를 올바르게 잡아내는 것 불가능


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X,y=make_moons(n_samples=1000,noise=0.05)
dbscan=DBSCAN(eps=0.05,min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [ ]:
dbscan.labels_

array([ 0, -1,  1,  0,  2,  1,  1,  2,  3,  3,  4, -1,  0, -1,  4,  1,  3,
        3,  5,  6,  2,  1,  7,  4,  1,  3, -1,  3,  1,  6,  1,  4, -1,  3,
        2,  3, -1,  5,  2,  4,  8,  1,  4,  2,  1,  3,  4,  5,  7,  3,  1,
       -1,  5,  3,  3,  4,  4,  3,  3,  8,  3,  2,  4,  1,  0, -1,  0,  3,
        3,  5,  9,  2,  5,  5,  1,  2,  1,  3, -1,  5,  2,  1,  2,  2,  8,
        6,  7,  2,  4,  2,  5,  2,  2,  1, -1,  2,  1,  2,  4,  5,  3,  2,
        3,  0, -1,  3,  0,  7,  4,  1,  3,  4,  0,  2,  4,  5,  1,  0,  2,
        4,  3,  2,  4, -1,  3,  4,  4,  2,  1,  3,  2,  3,  5,  0,  3,  8,
        2,  1,  1,  2,  5, -1,  2,  7,  7,  3,  3,  7,  7,  2,  9,  9, -1,
        3,  0,  7,  1,  1,  1,  2,  2,  4,  0,  4,  4, -1,  3,  4,  3,  1,
        7,  3,  8,  8, -1,  2,  1,  0,  0, -1,  1,  7, -1,  3,  4,  4,  1,
        3,  4,  4,  2,  4,  1,  0,  2,  0,  1,  3,  3,  0,  2,  3,  4,  4,
       -1, -1,  3,  8,  4,  3,  3,  1,  7,  1,  1,  0,  5,  3,  2,  0,  3,
        5,  4,  3,  8, -1

In [ ]:
len(dbscan.core_sample_indices_)

795

In [ ]:
dbscan.core_sample_indices_

array([  0,   2,   3,   4,   5,   7,   8,   9,  12,  14,  15,  16,  17,
        18,  19,  20,  22,  23,  24,  25,  27,  28,  29,  30,  33,  34,
        35,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  50,  52,  53,  54,  55,  57,  58,  60,  61,  62,  64,  66,
        67,  68,  71,  73,  74,  75,  76,  77,  79,  81,  82,  83,  84,
        85,  86,  87,  90,  91,  92,  93,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 105, 107, 108, 109, 110, 111, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130,
       131, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145,
       146, 147, 149, 150, 151, 153, 154, 156, 157, 159, 161, 163, 164,
       166, 168, 169, 170, 171, 172, 173, 175, 176, 177, 178, 181, 183,
       184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197,
       198, 199, 200, 202, 203, 206, 208, 209, 210, 211, 212, 213, 214,
       215, 216, 217, 219, 220, 221, 222, 223, 224, 226, 227, 22

In [ ]:
dbscan.components_

array([[ 0.97279995,  0.35510581],
       [-0.78499616,  0.67296112],
       [ 0.84715026,  0.57391434],
       ...,
       [-0.40082746,  0.86638853],
       [ 0.91351556,  0.30152899],
       [ 0.823884  ,  0.59026494]])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_,dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [ ]:
X_new=np.array([[-0.5,0],[0,0.5],[1,-0.1],[2,1]])
knn.predict(X_new)
knn.predict_proba(X_new)

array([[0.  , 0.68, 0.  , 0.16, 0.  , 0.  , 0.1 , 0.  , 0.06, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.56, 0.  , 0.  , 0.  , 0.  , 0.44, 0.  , 0.  ],
       [0.66, 0.  , 0.  , 0.04, 0.  , 0.12, 0.  , 0.  , 0.  , 0.18, 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [ ]:
y_dist,y_pred_idx=knn.kneighbors(X_new,n_neighbors=1)
y_pred=dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist>0.2]=-1
y_pred.ravel()

array([-1,  8,  0, -1])

### **9.1.7 다른 군집 알고리즘**
-**병합 군집**
- 클러스터 계층을 밑바닥부터 위로 쌓아 구성
- 대규모 샘플과 클러스터에 잘 확장되며 다양한 형태의 클러스터를 감지할 수 있음
- 특정 클러스터 개수를 선택하는 데 도움이 되는 유용한 클러스터 트리를 만들 수 있음
- 연결 행렬을 이용해 대규모 데이터셋으로 확장

-**BIRCH**
- 대규모 데이터셋을 위해 고안
- 특성 개수가 20개 이하라면, k-평균보다 빠르고 비슷한 결과를 만들 수 있음
- 훈련 과정에서 새로운 샘플을 클러스터에 빠르게 할당할 수 있는 정보를 담은 트리 구조 생성
- 트리에 모든 샘플 저장X

-**평균-이동**
- 알고리즘은 먼저 각 샘플을 중심으로 하는 원을 그림 > 원마다 안에 포함된 모든 샘플의 평균을 구함 > 원의 중심을 평균점으로 이동 > 모든 원이 움직이지 않을 때까지 계속  
- 지역의 최대 밀도를 찾을 때까지 높은 쪽으로 원 이동  
- 동일한 지역에 안착한 원에 있는 모든 샘플은 동일한 클러스터가 됨
- DBSCAN과 유사하게 모양이나 개수에 상관없이 클러스터를 찾을 수 있음
-하이퍼 파라미터: bandwidth
- 클러스터 내부 밀집도가 불균형할 때 여러 개로 나누는 경향
- 대규모 데이터셋에는 적합X

-**유사도 전파**
- 투표 방식 사용
- 자신을 대표할 수 있는 비슷한 샘플에 투표
- 알고리즘이 수렴하면, 각 대표와 투표한 샘플이 클러스터 형성  
- 크기가 다른 여러 개의 클러스터를 감지할 수 있음
- 대규모 데이터셋에는 적합X  
**계산복잡도가 O(m^2)이면 대규모 적합 안되는거임..?

-**스펙트럼 군집**
- 샘플 사이의 유사도 행렬을 받아 저차원 임베딩 생성  
- 저차원 공간에서 또 다른 군집 알고리즘 사용  
- 복잡한 클러스터 구조를 감지하고 그래프 컷을 찾는데 사용가능
- 샘플 개수가 많으면 잘 적용되지 않고 클러스터의 크기가 매우 다르면 잘 동작X

## **9.2 가우시안 혼합(GMM)**
-샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델  
-하나의 가우시안 분포에서 생성된 모든 샘플은 하나의 클러스터 형성(일반적으로 타원형)  
-샘플이 주어지면, 가우시안 분포 중 하나에서 생성되었다는 것은 알지만, 어떤 분포인지 파라미터는 무엇인지 알지 못함  
-GaussianMixture  
  - 기댓값-최대화(EM) 알고리즘 사용
  - (하드군집)새로운 샘플을 가장 비슷한 클러스터에 손쉽게 할당: predict()
  - (소프트군집)특정 클러스터에 속할 확률을 예측할 수 있음: predict_proba()  
-생성모델(모델에서 새로운 모델 생성 가능)


In [ ]:
from sklearn.mixture import GaussianMixture
gm=GaussianMixture(n_components=3,n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [ ]:
gm.weights_

array([0.19602249, 0.62391876, 0.18005874])

In [ ]:
gm.means_

array([[-0.7593324 ,  0.54244533],
       [ 0.51999812,  0.2310231 ],
       [ 1.79871057, -0.01185186]])

In [ ]:
gm.covariances_

array([[[ 0.04776042,  0.05839525],
        [ 0.05839525,  0.08641268]],

       [[ 0.19779982, -0.12599027],
        [-0.12599027,  0.29892629]],

       [[ 0.03277323,  0.04653762],
        [ 0.04653762,  0.0809964 ]]])

In [ ]:
gm.converged_

True

In [ ]:
gm.n_iter_

21

In [ ]:
gm.predict(X)

array([1, 1, 0, 1, 2, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 2, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 1, 2,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 0, 2, 0, 1, 1, 1, 2, 0, 2, 2, 1, 0, 1, 2,
       1, 2, 2, 2, 2, 0, 2, 2, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 2, 0, 1, 0, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 1, 2, 0, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1,
       1, 1, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 2, 2,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 1, 0, 1, 2, 1, 0, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 0,
       1, 2, 2, 1, 2, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1,
       1, 0, 2, 0, 1, 0, 2, 1, 1, 1, 0, 2, 0, 1, 1, 0, 1, 1, 2, 1, 2, 1,
       2, 2, 1, 1, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 2,

In [ ]:
gm.predict_proba(X)

array([[1.94894654e-090, 1.00000000e+000, 6.14065949e-039],
       [1.70095501e-047, 1.00000000e+000, 4.63209528e-091],
       [9.90656705e-001, 9.34329485e-003, 1.62745224e-320],
       ...,
       [7.78954022e-001, 2.21045978e-001, 3.21636719e-265],
       [2.19556344e-088, 1.00000000e+000, 4.05952461e-041],
       [7.03781575e-063, 1.00000000e+000, 1.23638462e-063]])

In [ ]:
X_new,y_new=gm.sample(6)
X_new

array([[-0.70974807,  0.74077381],
       [ 0.53441708,  0.517823  ],
       [ 0.57340254, -0.54911951],
       [-0.10012377, -0.45100132],
       [ 1.72985425,  0.1564096 ],
       [ 1.7349006 , -0.15820217]])

In [ ]:
y_new

array([0, 1, 1, 1, 2, 2])

In [ ]:
#모델의 밀도 추정
gm.score_samples(X)

array([-1.64661302e+00, -1.23198579e+00, -7.17574665e-01, -1.70469755e+00,
       -3.47499638e-01, -7.07207183e-01, -1.88137620e+00, -4.25174839e-01,
       -1.64975665e+00, -1.53537765e+00, -1.81730153e+00, -1.59292999e+00,
       -1.51516525e+00, -1.32009441e+00, -1.70048403e+00, -3.05390253e-01,
       -1.88650226e+00, -1.77949677e+00, -2.36840294e+00, -2.97817450e+00,
       -8.12081261e-01, -7.02413631e-01, -1.77615431e+00, -1.94294615e+00,
       -1.28382128e-01, -1.59503279e+00, -1.51188766e+00, -1.57627336e+00,
       -1.24715071e-01, -2.78604451e+00, -3.24252941e-01, -1.59067505e+00,
       -1.18770672e+00, -1.44495232e+00,  9.53094876e-02, -1.65245278e+00,
       -2.14210845e+00, -1.88274108e+00, -8.56438011e-01, -1.57142659e+00,
       -1.42894206e+00, -5.70498218e-01, -2.14375966e+00, -5.02587013e-01,
       -6.49640018e-01, -1.42475379e+00, -1.72455785e+00, -2.14468512e+00,
       -1.64023495e+00, -1.58265995e+00, -1.18675683e+00, -2.31438174e+00,
       -2.04603535e+00, -

-특성이나 클러스터가 많거나 샘플이 적을 때는 EM이 최적으로 솔루션으로 수렴하기 어려움  
-> 알고리즘이 학습할 파라미터 개수 제한 위해 클러스터의 모양과 방향의 범위 제한
- covariance_type  
  - "spherical": 모든 클러스터 원형, 지름(분산) 다를 수 있음
  - "diag": 클러스터는 크기에 상관없이 어떤 타원형도 가능, 타원의 축은 좌표 축과 나란해야함(공분산 행렬이 대각 행렬)
  - "tied": 모든 클러스터가 동일한 타원 모양, 크기 방향을 가져야 함(동일한 공분산 행렬 공유)
  -"full": 기본값, 각 클러스터는 모양, 크기, 방향에 제약이 없음  
-이상치 탐지에도 사용가능

### **9.2.1 가우시안 혼합을 사용한 이상치 탐지**
-보통과 많이 다른 샘플 감지  
-밀도가 낮은 지역에 있는 모든 샘플=이상치
-사용할 밀도 임곗값 설정해야함  
-가우시안 혼합 모델은 이상치를 포함해 모든 데이터에 맞추려고 하기 때문에 이상치 제거 필요  


In [ ]:
densities=gm.score_samples(X)
density_threshold=np.percentile(densities,4)
anomalies=X[densities<density_threshold]

### **9.2.2 클러스터 개수 선택**
-BIC: Bayesian informationcriterion  
-AIC: Akaike information criterion  
-둘의 선택이 다를 경우, BIC가 선택한 모델이 더 간단한 경향이 있음


In [ ]:
gm.bic(X)

2767.2429203559227

In [ ]:
gm.aic(X)

2683.8110806132263

### **9.2.3 베이즈 가우시안 혼합 모델**
-최적의 클러스터 개수를 수동으로 찾지 않고 불필요한 클러스터의 가중치를 0으로 만드는 클래스  
-자동으로 불필요한 클러스터 제거

In [ ]:
from sklearn.mixture import BayesianGaussianMixture
bgm=BayesianGaussianMixture(n_components=10,n_init=10)
bgm.fit(X)
np.round(bgm.weights_,2)

array([0.16, 0.14, 0.09, 0.14, 0.11, 0.18, 0.07, 0.11, 0.  , 0.  ])

### **9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘**
- PCA: 샘플의 재구성 오차와 이상치의 재구성 오차 비교  
- Fast-MCD  
  - 데이터셋을 정제할 때 사용
  - 샘플(정상치)이 하나의 가우시안 분포에서 생성되었다고 가정
  - 가우시안 분포에서 생성되지 않은 이상치로 데이터셋이 오염되었다고 가정
  - 이상치로 의심되는 샘플 무시
  - 타원형 잘 추정, 이상치 잘 구분
- 아이솔레이션 포레스트
  - 고차원 데이터셋에서 이상치 감지를 위한 효율적인 알고리즘
  - 무작위로 성장한 결정 트리로 구성된 랜덤 포레스트 생성
- LOF
  - 이상치 탐지에 좋음
  - 주어진 샘플 주위의 밀도와 이웃 주위의 밀도 비교
- one-class SVM
  - 특이치 탐지에 잘 맞음
  - 고차원 데이터셋에 잘 작동하지만, 대규모 데이터셋으로 확장은 어려움